## Estimation of the effects following Aakvik et al. (2005)

In [1]:
%reload_ext cython

### First some packages need to be imported

In [2]:
%%cython
import numpy as np
import time
import scipy
from scipy import integrate
from scipy import stats
from numpy import arange
from scipy import optimize
import pandas as pd
import pybobyqa
np.random.seed(11813242)
  

17839.86681768303
28.632741451263428


/home/christian/anaconda3/lib/python3.7/site-packages/pybobyqa/solver.py:774: RuntimeWarning: x0 above upper bound, adjusting
  warnings.warn("x0 above upper bound, adjusting", RuntimeWarning)


****** Py-BOBYQA Results ******
Solution xmin = [0.70443884 0.71632578 0.71588429 1.01436051 0.95059334]
Objective value f(xmin) = 16323.4077
Needed 85 objective evaluations (at 85 points)
Approximate gradient = [-7.57694829e-05  1.00917344e-04 -4.23307732e-05 -6.11028365e-05
 -6.41631294e-05]
Approximate Hessian = [[ 25922.77259987  -6168.90522528  -3898.97858209 -12088.88721011
    1229.55783229]
 [ -6168.90522528  49271.7336352   13076.36927422   -517.44787323
   -2222.18742871]
 [ -3898.97858209  13076.36927422  15197.48653618   8006.23062428
    9072.84515531]
 [-12088.88721011   -517.44787323   8006.23062428   3216.0109774
    3426.30849068]
 [  1229.55783229  -2222.18742871   9072.84515531   3426.30849068
    3167.98438818]]
Exit flag = 0
Success: rho has reached rhoend
******************************

2174.9519739151


In [ ]:
#define probabilities needed to calculate ML function
np.random.seed(11813242)
def Pr_Y(X,beta0,alpha0,beta1,alpha1,x,D,Y ):
    if D == 1 and Y == 1:
        a = scipy.stats.norm.cdf(beta0*X+alpha0*x,0,1)
    elif D == 1 and Y == 0:
        a = 1 - scipy.stats.norm.cdf(beta0*X+alpha0*x,0,1)
    elif D == 0 and Y == 1:
        a = scipy.stats.norm.cdf(beta1*X + alpha1*x,0,1)
    else:
        a = 1 - scipy.stats.norm.cdf(beta1*X + alpha1*x,0,1)
    return a
#only calcualte the probability for a limited range in order to use the gaussian quadrature for the integration as Aakvik et al. does
def Pr_Y_lim(X,beta0,alpha0,beta1,alpha1,x,D,Y):
    ranges = [x<-10,x>10]
    values = [0,0,Pr_Y(X,beta0,alpha0,beta1,alpha1,x,D,Y)]
    return np.piecewise(x,ranges,values)

def  Pr_D(Z,beta_D, x, D):
    if D == 1:
        b = scipy.stats.norm.cdf(beta_D*Z+x,0,1)
    else:
        b = 1 - scipy.stats.norm.cdf(beta_D*Z+x,0,1)
    return b

def Pr_D_lim(Z,beta_D, x, D):
    ranges = [x<-10,x>10]
    values = [0,0,Pr_D(Z,beta_D, x, D )]
    return np.piecewise(x,ranges,values)

    
def alltogether(Z,X, beta_D,beta0,alpha0,alpha1,beta1,x,D,Y):
    a = Pr_D(Z,beta_D,x,D)
    b = Pr_Y(X,beta0,alpha0,beta1,alpha1,x,D,Y)
    c = scipy.stats.norm.pdf(x)
    return a*b*c
    #return a*b
#define maximum likelihood function
def trial(Z,X,params,D,Y):
    beta_D,beta0, beta1, alpha0,alpha1, = params
    a = scipy.integrate.fixed_quad(lambda x: alltogether(Z,X,beta_D,beta0,alpha0,alpha1,beta1,x,D,Y), -10,10)[0]
    return -np.log(a)
#def trial_2(Z, X,params, D,Y):
#    beta_D,beta0,alpha0,alpha1, beta1 = params
#    a = scipy.integrate.fixed_quad(lambda x: alltogether_2(Z,X,beta_D,beta0,alpha0,alpha1,beta1,x,D,Y),-10,10)[0]
#    return -np.log(a)





In [ ]:
#simulation of the model with altered structure according to Aakvik et al. 
n = 50000
mean = [0,0,0]
cov = [[1,0,0],[0,1,0],[0,0,1]]
obs = np.random.multivariate_normal(mean,cov,n)
Z = obs[:,0]
X = obs[:,1]
theta_2 = obs[:,2]
unobs = np.random.multivariate_normal(mean,cov,n)
U_D = unobs[:,0]
U_1 = unobs[:,1]
U_0 = unobs[:,2]
beta_D = 1
beta0 = 1
beta1 = 1
alpha0 = 1
alpha1 = 1
D_star = Z*beta_D + theta_2 - U_D
df = pd.DataFrame(data=D_star, columns=['D_star'])
df['D'] = 0
df
df.loc[df['D_star'] > 0, 'D'] = 1
df[ 'X'] = X
df['Z'] = Z
df['theta_2'] = theta_2
df['U_D'] = U_D
df['U_1'] = U_1
df['U_0'] = U_0
df['Y1_star'] = X*beta1 + alpha1*theta_2 - U_1
df['Y1'] = 0
df.loc[df['Y1_star'] > 0,'Y1'] = 1
df['Y0_star'] = X*beta0 + alpha0*theta_2 - U_0
df['Y0'] = 0
df.loc[df['Y0_star'] > 0,'Y0'] = 1
df['Y'] = df['Y1']
df.loc[df['D'] == 0, 'Y'] = df['Y0']
D = np.asarray(df.D)
Y = np.asarray(df.Y)
array = (Z,X,D,Y)


In [ ]:
#calculate object to be optimized
array_T = np.transpose(array)
range_target = np.arange(0,n,1)
def target(params):
    obj = []
    for x in range_target:
        t = trial(array_T[x,0],array_T[x,1],params,array_T[x,2],array_T[x,3])
        obj.append(t)
    return np.sum(obj)

In [ ]:
#test various solution algorithms 
from scipy.optimize import Bounds
#bnds = Bounds([0,0,0,0,0], [4,4,4,4,4])
start = time.time()
x0 = np.array([1,1,0.99,1,1])
bnds = Bounds([-1,-1,-1,-1,-1],[4,4,4,4,4])
lower = np.array([-4,-4,-4,-4,-4])
upper = np.array([1,4,4,4,4])
#cons = scipy.optimize.LinearConstraint(([beta_D],[beta0],[alpha0], [alpha1], [beta1]),(1,1,1,1,1),(1,1,1,1,1))
rranges = (slice(-2, 2, 0.5), slice(-2, 2, 0.5),slice(-2, 2, 1), slice(-2, 2, 1),slice(-2, 2, 1))
print(pybobyqa.solve(target, x0, bounds=(lower, upper),maxfun = 8000))
#a = scipy.optimize.minimize(target, x0, method='SLSQP',bounds=bnds,options={'maxiter':2000})
#b = scipy.optimize.least_squares(target,x0,bounds=(lower,upper))
#c = scipy.optimize.brute(target,rranges)
#d = scipy.optimize.basinhopping(target,x0)
#print(a)
#print(b)
#print(c)
end = time.time()
print(end - start)

In [6]:
#define ATE  from  Aakvik et al. as example
import scipy
from scipy import integrate
from scipy import stats
def ATE(X):
    beta1 =1
    beta0 =1
    alpha0 =1
    alpha1=1
    a = scipy.integrate.fixed_quad(lambda x: [scipy.stats.norm.cdf(beta1*X + alpha0*x,0,1) - scipy.stats.norm.cdf(beta0*X + alpha0*x,0,1)]*scipy.stats.norm.pdf(x),-10,10)[0]
    return a


array([0.])